In [1]:
import sys
!{sys.executable} -m pip install gdown opencv-python Pillow scikit-learn seaborn pycocotools matplotlib ultralytics torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 16.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 23.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 14.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 18.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 22.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 21.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25/25 [ultralytics] [ultralytics]]]


In [2]:
import os
import yaml
from ultralytics import YOLO
import gdown
import zipfile
import torch
import shutil

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# --- ¡IMPORTANTE! REVISA Y MODIFICA ESTA INFORMACIÓN ---
# 1. PEGA AQUÍ EL ID DE TU ARCHIVO ZIP DEL DATASET GRANDE (15 mil imágenes)
FILE_ID_DATASET = '18IaSltzRrYrHICOoGdnB55hDgp_pB3f7' # <--- ¡REVISA QUE ESTE SEA EL ID CORRECTO!

# 2. NOMBRES PARA LOS ARCHIVOS Y CARPETAS DE SALIDA
DATASET_ZIP_FILENAME = 'dataset_principal.zip'
EXTRACT_TO_FOLDER = './dataset_entrenamiento'
RESULTS_DIR = './resultados_completos_sensible'
FINAL_RESULTS_ZIP_NAME = 'resultados_finales_detector_sensible.zip'

# --- Verificación de GPU ---
if torch.cuda.is_available():
    print(f"\n✅ GPU detectada: {torch.cuda.get_device_name(0)}")
else:
    print("\n❌ ADVERTENCIA: No se detectó una GPU. El entrenamiento será extremadamente lento.")
    print("Asegúrate de haber seleccionado una imagen de Docker con soporte para CUDA/PyTorch en vast.ai.")



✅ GPU detectada: NVIDIA GeForce RTX 5060 Ti


In [4]:
# ===================================================================
# PASO 2: DESCARGA Y PREPARACIÓN DEL DATASET
# ===================================================================
try:
    print(f"\nDescargando dataset principal...")
    gdown.download(id=FILE_ID_DATASET, output=DATASET_ZIP_FILENAME, quiet=False)
    print("Descomprimiendo dataset principal...")
    os.makedirs(EXTRACT_TO_FOLDER, exist_ok=True)
    with zipfile.ZipFile(DATASET_ZIP_FILENAME, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_TO_FOLDER)
    os.remove(DATASET_ZIP_FILENAME)
    print("Dataset principal listo.")
except Exception as e:
    print(f"ERROR al procesar el dataset principal: {e}")
    raise SystemExit("Deteniendo ejecución.")


Descargando dataset principal...


Downloading...
From (original): https://drive.google.com/uc?id=18IaSltzRrYrHICOoGdnB55hDgp_pB3f7
From (redirected): https://drive.google.com/uc?id=18IaSltzRrYrHICOoGdnB55hDgp_pB3f7&confirm=t&uuid=10f8f518-1826-4231-8056-bbe193fb0b73
To: /Modelo yolov8nano/dataset_principal.zip
100%|██████████| 840M/840M [00:20<00:00, 40.2MB/s] 


Descomprimiendo dataset principal...
Dataset principal listo.


In [5]:
# --- Unificar Clases ---
def merge_classes_in_labels(directory):
    if not os.path.isdir(directory): return
    for label_file in os.listdir(directory):
        if label_file.endswith(".txt"):
            filepath = os.path.join(directory, label_file)
            if os.path.getsize(filepath) > 0:
                with open(filepath, 'r') as f: lines = f.readlines()
                new_lines = []
                for line in lines:
                    parts = line.strip().split()
                    if parts:
                        parts[0] = '0'
                        new_lines.append(" ".join(parts))
                with open(filepath, 'w') as f: f.write("\n".join(new_lines))

print("\nModificando etiquetas para unificar en una sola clase...")
train_labels_dir = os.path.join(EXTRACT_TO_FOLDER, 'train', 'labels')
valid_labels_dir = os.path.join(EXTRACT_TO_FOLDER, 'valid', 'labels')
test_labels_dir = os.path.join(EXTRACT_TO_FOLDER, 'test', 'labels')

merge_classes_in_labels(train_labels_dir)
merge_classes_in_labels(valid_labels_dir)
merge_classes_in_labels(test_labels_dir)
print("Etiquetas modificadas con éxito.")

# --- CORRECCIÓN AUTOMÁTICA DEL ARCHIVO data.yaml ---
yaml_path = None
# Buscamos el archivo 'data.yaml'
for root, dirs, files in os.walk(EXTRACT_TO_FOLDER):
    if 'data.yaml' in files:
        yaml_path = os.path.join(root, 'data.yaml')
        break

if yaml_path:
    print(f"\nArchivo 'data.yaml' encontrado en: {yaml_path}")

    with open(yaml_path, 'r') as f: data_yaml = yaml.safe_load(f)

    # El directorio base es donde está el propio archivo .yaml
    dataset_base_path = os.path.dirname(yaml_path)

    # --- CORRECCIÓN CLAVE ---
    # Borramos cualquier ruta 'path' que pudiera causar problemas
    data_yaml.pop('path', None)
    # Escribimos las rutas ABSOLUTAS y CORRECTAS para el entorno actual
    data_yaml['train'] = os.path.abspath(os.path.join(dataset_base_path, 'train/images'))
    data_yaml['val'] = os.path.abspath(os.path.join(dataset_base_path, 'valid/images'))
    if os.path.exists(os.path.join(dataset_base_path, 'test')):
        data_yaml['test'] = os.path.abspath(os.path.join(dataset_base_path, 'test/images'))

    # Actualizar para una sola clase
    data_yaml['nc'] = 1
    data_yaml['names'] = ['hez']

    with open(yaml_path, 'w') as f:
        yaml.dump(data_yaml, f)

    print("Archivo 'data.yaml' corregido y actualizado.")
else:
    raise SystemExit("ERROR: No se pudo encontrar 'data.yaml'.")


Modificando etiquetas para unificar en una sola clase...
Etiquetas modificadas con éxito.

Archivo 'data.yaml' encontrado en: ./dataset_entrenamiento/data.yaml
Archivo 'data.yaml' corregido y actualizado.


In [6]:
# ===================================================================
# PASO 4: ENTRENAR EL MODELO
# ===================================================================
model = YOLO('yolov8n.pt')

print("\nIniciando entrenamiento del modelo...")
model.train(
    data=yaml_path,
    epochs=100,
    imgsz=640,
    project=RESULTS_DIR,
    name='entrenamiento_detector_sensible'
)
print("\n--- ENTRENAMIENTO FINALIZADO ---")


Iniciando entrenamiento del modelo...
Ultralytics 8.3.176 🚀 Python-3.12.11 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5060 Ti, 15856MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./dataset_entrenamiento/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=entrenamiento_detector_sensible, nbs=64, nms=False, opset=None, optimize=False, optimizer=

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 129 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1987.9±607.0 MB/s, size: 56.4 KB)


train: Scanning /Modelo yolov8nano/dataset_entrenamiento/train/labels... 14115 images, 3 backgrounds, 0 corrupt: 100%|██████████| 14115/14115 [00:05<00:00, 2671.32it/s]


...


    100/100      3.69G       1.23     0.4879        1.6          3        640: 100%|██████████| 883/883 [01:08<00:00, 12.90it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:03<00:00, 10.25it/s]

                   all       1037       1063      0.942      0.917      0.953      0.529

100 epochs completed in 2.009 hours.


Optimizer stripped from resultados_completos_sensible/entrenamiento_detector_sensible/weights/last.pt, 6.3MB
Optimizer stripped from resultados_completos_sensible/entrenamiento_detector_sensible/weights/best.pt, 6.3MB

Validating resultados_completos_sensible/entrenamiento_detector_sensible/weights/best.pt...
Ultralytics 8.3.176 🚀 Python-3.12.11 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5060 Ti, 15856MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:03<00:00,  8.58it/s]


                   all       1037       1063      0.945      0.915      0.956      0.541
Speed: 0.2ms preprocess, 0.9ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to resultados_completos_sensible/entrenamiento_detector_sensible

--- ENTRENAMIENTO FINALIZADO ---


In [7]:
# ===================================================================
# PASO 5: EVALUAR (TESTEAR) EL MODELO ENTRENADO
# ===================================================================
print("\n--- INICIANDO EVALUACIÓN EN EL CONJUNTO DE PRUEBA (TEST) ---")
best_model_path = os.path.join(RESULTS_DIR, 'entrenamiento_detector_sensible/weights/best.pt')

if os.path.exists(best_model_path) and 'test' in data_yaml:
    model = YOLO(best_model_path)
    model.val(
        data=yaml_path,
        split='test',
        project=RESULTS_DIR,
        name='evaluacion_detector_sensible'
    )
    print("\n--- EVALUACIÓN FINALIZADA ---")
else:
    print("Advertencia: No se evaluó el modelo.")


--- INICIANDO EVALUACIÓN EN EL CONJUNTO DE PRUEBA (TEST) ---
Ultralytics 8.3.176 🚀 Python-3.12.11 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5060 Ti, 15856MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1435.5±581.4 MB/s, size: 52.7 KB)


val: Scanning /Modelo yolov8nano/dataset_entrenamiento/test/labels... 1037 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1037/1037 [00:00<00:00, 2104.30it/s]

val: New cache created: /Modelo yolov8nano/dataset_entrenamiento/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 65/65 [00:04<00:00, 14.10it/s]


                   all       1037       1057      0.966      0.946      0.977      0.555
Speed: 0.4ms preprocess, 1.6ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to resultados_completos_sensible/evaluacion_detector_sensible

--- EVALUACIÓN FINALIZADA ---


In [8]:
# ===================================================================
# PASO 6: EMPAQUETAR LOS RESULTADOS PARA LA DESCARGA
# ===================================================================
if os.path.exists(RESULTS_DIR):
    print(f"\nComprimiendo la carpeta de resultados '{RESULTS_DIR}'...")
    with zipfile.ZipFile(FINAL_RESULTS_ZIP_NAME, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(RESULTS_DIR):
            for file in files:
                zipf.write(os.path.join(root, file),
                           os.path.relpath(os.path.join(root, file),
                                           os.path.join(RESULTS_DIR, '..')))

    print("\n" + "="*50)
    print("¡PROCESO COMPLETADO!")
    print(f"Tus resultados (entrenamiento y testeo) están en el archivo:")
    print(f"===> {FINAL_RESULTS_ZIP_NAME} <===")
    print("Ahora puedes descargar este único archivo desde la interfaz de vast.ai.")
    print("="*50)
else:
    print("ERROR: No se encontró la carpeta de resultados para empaquetar.")


Comprimiendo la carpeta de resultados './resultados_completos_sensible'...

¡PROCESO COMPLETADO!
Tus resultados (entrenamiento y testeo) están en el archivo:
===> resultados_finales_detector_sensible.zip <===
Ahora puedes descargar este único archivo desde la interfaz de vast.ai.
